In [66]:
# !pip install nltk
!pip install wikipedia

In [67]:
import nltk
import random
import string
import re
import unicodedata
import wikipedia as wk
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity , linear_kernel

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [68]:
data =open('/content/drive/MyDrive/Chatbot/HR.txt','r',errors='ignore')
raw = data.read()
raw = raw.lower()

In [69]:
# raw[:1000]

In [70]:
sent_tokens = nltk.sent_tokenize(raw)

In [71]:
# sent_tokens

In [72]:
def Normalize(text):
  # Filtering Words
  remove_punct_dict = dict((ord(punct) , None) for punct in string.punctuation)  # Remove Punctuations
  # Word Tokenization
  word_token = nltk.word_tokenize(text.lower().translate(remove_punct_dict))
  # Remove ASCII
  new_words = []
  for word in word_token:
    new_word = unicodedata.normalize('NFKD',word).encode('ascii','ignore').decode('utf-8','ignore')
    new_words.append(new_word)
  # Remove Tags
  rmv = []
  for w in new_words:
    text = re.sub("&lt;/?.*?&gt;","&lt;&lt;" , w)
    rmv.append(text)
  # Pos Tagging & Lemmatization
  tag_map = defaultdict(lambda:wn.NOUN)
  tag_map['J'] = wn.ADJ
  tag_map['V'] = wn.VERB
  tag_map['R'] = wn.ADV
  lmtzr = WordNetLemmatizer()
  lemma_list = []
  rmv = [i for i in rmv if i]
  for token , tag in nltk.pos_tag(rmv):
    lemma = lmtzr.lemmatize(token , tag_map[tag[0]])
    lemma_list.append(lemma)
  return lemma_list

In [73]:
welcome_input = ("hello" , "hi" , "greetings" , "sup" , "what's up"  , "hey")
welcome_response = ["hi" , "hey" , "nods" , "hi there" , "hello" , "I am glad! You are talking to me"]

def welcome(user_response):
  for word in user_response.split():
    if word.lower() in welcome_input:
      return random.choice(welcome_response)

In [74]:
def generateResponse(user_response):
  robo_response = '  '
  sent_tokens.append(user_response)
  TfidfVec = TfidfVectorizer(tokenizer=Normalize , stop_words='english')
  tfidf = TfidfVec.fit_transform(sent_tokens)

  vals = linear_kernel(tfidf[-1],tfidf)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]
  if (req_tfidf ==0) or "tell me about" in user_response:
    print("Checking Wikipedia ...")
    if user_response:
      robo_response = wikipedia_data(user_response)
      return robo_response
  else:
    robo_response = robo_response + sent_tokens[idx]
    return robo_response

In [75]:
def wikipedia_data(input):
  reg_ex = re.search('Tell me About(.*)' , input)
  try:
    if reg_ex:
      topic = reg_ex.group(1)
      wiki = wk.summary(topic , sentences = 3)
      return wiki
  except Exception as e:
    print("No Content has been Found")

In [76]:
flag = True
print("My name is Chatter and I am a Chatbot")
while(flag==True):
  user_response = input()
  user_response = user_response.lower()
  if (user_response not in ["bye" , "shutdown" , "exit" , "quit"]):
    if (user_response=="thanks" or user_response=="thankyou"):
      flag = False
      print("ChatterBot : You're Welcome....")
    else:
      if (welcome(user_response)!= None):
        print("ChatterBot : "+welcome(user_response))
      else:
        print("ChatterBot : ",end ="")
        print(generateResponse(user_response))
        sent_tokens.remove(user_response)
  else:
    flag = False
    print("ChatterBot : Bye!!!! ")

My name is Chatter and I am a Chatbot
zunaisha
ChatterBot : Checking Wikipedia ...
None
tell me about angelina jolie
ChatterBot : Checking Wikipedia ...
None
bye
ChatterBot : Bye!!!! 
